In [13]:
import urllib.request as req #引入urllib模組中函式名稱為request的函式為req
import bs4 #引入bs4模組
import re #引入re模組
#定義函式getData，用以取得每一頁所有標題、標籤及內文。參數為網址
def getData(url,count,pages):
    #建立一個Request物件，附加Request Headers的資訊
    request = req.Request(url,headers = {
        "User-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.0.0 Safari/537.36"
    })
    
    #打開request
    with req.urlopen(request) as response:
        data = response.read().decode("utf-8")

    root = bs4.BeautifulSoup(data,"html.parser") #讓BeautifulSoup解析HTML格式文件
    all_titles = root.find_all("h3",class_="qa-list__title") #以列表形式找出所有class_=title的h3標籤


    #開啟JSONcrawler.json並用a模式保留原本資料並將新的資料寫入
    with open("JSONcrawler.json",mode = "a",encoding = "utf-8") as ftext:
        #此迴圈會找出該頁所有標題、標籤、詢問時間、瀏覽次數及內文
        for title in all_titles:
            title.a.string = re.sub(r"\"","'",title.a.string) #利用regex將 " 改成 ' ，以免語法錯誤
            title.a.string = re.sub(r"\\"," BACKSLASH ",title.a.string) #利用regex將 \ 改成 BACKLASH ，以免語法錯誤
            ftext.write("\n   {\n") #寫入與JSON檔的檔案格式有關的detail
            ftext.write("   \"標題\":"+"\""+title.a.string+"\","+"\n") #將內文的標題寫入JSON檔裡
            print("標題 : "+title.a.string)
            
            ftext.write("   \"標籤\":[") #寫入與JSON檔的檔案格式有關的detail

            url2 = title.a["href"] #將內文的網址存到url2裡

            #建立一個Request物件，附加Request Headers的資訊
            request2 = req.Request(url2,headers = {
                "User-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.0.0 Safari/537.36"
            })

            #打開request2
            with req.urlopen(request2) as response2:
                data2 = response2.read().decode("utf-8")
            
            root2 = bs4.BeautifulSoup(data2,"html.parser") #讓BeautifulSoup解析HTML格式文件
            contents = root2.find("div",class_="markdown__style").text #找出內文
            tags = root2.find("div",class_="qa-header__tagGroup") #以列表形式找出所有 class_=qa-header__tagGroup 的 div 標籤
            time_and_view = root2.find("div",class_="qa-header__info").text.split() #以列表形式找出所有 class_=qa-header__info 的 div 標籤
            
            #利用regex將會影響JSON的符號做替換
            contents = re.sub(r"\"","'",contents)
            contents = re.sub(r"\\"," BACKSLASH ",contents)
            contents = re.sub(r"\n","EOL",contents)
            contents = re.sub(r"\t","    ",contents)

            set_of_tags = set(tags.text.split("\n")) #利用集合將重複出現的標籤強制讓它出現一次
            #將集合內的資料重新裝回list內
            list_of_tags = []
            for i in set_of_tags:
                list_of_tags.append(i)
            
            #將標籤寫進JSON檔
            for tag in list_of_tags[1:]:
                tag = re.sub(r"\\"," BACKSLASH ",tag)
                if (tag!=list_of_tags[-1]):
                    ftext.write("\"" + tag + "\",")
                else:
                    ftext.write("\"" + tag + "\"],\n")
            
            print("標籤 :",end = " ")
            for tag in list_of_tags[1:]:
                print(tag,end = " ")
            
            ftext.write("   \"詢問時間\":\""+time_and_view[1]+"\",\n") #將詢問時間寫入JSON檔
            print("\n詢問時間 : "+time_and_view[1])
            ftext.write("   \"瀏覽次數\":\""+time_and_view[-2]+"\",\n") #將瀏覽次數寫入JSON檔
            print("瀏覽次數 : "+time_and_view[-2])

            #將內文寫入JSON檔
            if ( (count==pages-1) and (title.a.string == all_titles[-1].a.string)):
                ftext.write("   \"內文\":\""+contents+"\"\n") #將內文寫入content.txt裡
                ftext.write("   }]\n}")
                print("內文 : "+contents)
            else:
                ftext.write("   \"內文\":\""+contents+"\"\n")
                ftext.write("   },\n")
                print("內文 : "+contents+"\n\n")


    #抓取下一頁的連結
    nextLink = root.find("a",string="下一頁") #找到內文是 下一頁 的標籤a
    return nextLink["href"] #回傳下一頁的網址


In [14]:
#寫入與JSON檔的檔案格式有關的detail
with open("JSONcrawler.json",mode = "a",encoding = "utf-8") as ftext:
    ftext.write("{\n")
    ftext.write("   \"Article\":[")
pageURL = "https://ithelp.ithome.com.tw/" #將起始網頁的網址存到pageURL裡
count = 0 #count用來表示想抓取幾頁
pages = int(input("Please enter how many pages you want to crawl:"))
#重複抓取每頁資料
while count<pages:
    pageURL = getData(pageURL,count,pages) #將 return 回來的網址覆蓋到pageURL上，以便重複利用
    count += 1 #執行完一次後增加1，以確保能抓到預期的頁數

Please enter how many pages you want to crawl:1
標題 : 請問java如何獲取字串裡的數字?
標籤 : android java 
詢問時間 : 2022-11-15
瀏覽次數 : 26
內文 : EOL請問java如何獲取字串裡的數字?EOL字串中有英文和數字和＜＞＝符號夾雜，我只要將數字取出放到ｉｎｔ變數裡面請問要如何做？EOL拜託各位了　我上網找只找到一堆不能用的方法　＝　＝　EOL（另外是android studio的ｊａｖａ喔）EOL


標題 :  ubuntu的掛載問題
標籤 : ubuntu 22.04 ubuntu linux boot 
詢問時間 : 2022-11-15
瀏覽次數 : 63
內文 : EOL根這個狀況蠻類似的EOLhttps://stackoverflow.com/questions/68124511/booting-error-dev-sda2-clean-files-blocksEOL但是我的是 /dev/sdb2EOL雖然這樣開的了機，但是，銀幕亮度卻沒辦法條。EOL請問有什麼辦法可以解決，還有請教一下原因，謝謝。（上面的方法已經試過了）EOL


標題 : Postman Mock Server 可否帶入 Body 參數
標籤 : #postman body mockserver 
詢問時間 : 2022-11-15
瀏覽次數 : 69
內文 : EOL自己實作與查看網路上的文章後，EOL發現 Postman 的 MockServer 好像只能判斷 QueryString 參數，EOL而從 Header、Body 內帶入的參數就完全沒判斷，EOL若有後者的需求，是否就只能自己建立 MockServer 了呢？EOL


標題 : 試用Apache Jmeter 5.5時，Java不認識https？
標籤 : apache jmeter 
詢問時間 : 2022-11-15
瀏覽次數 : 85
內文 : EOLjava.net.MalformedURLException: unknown protocol: httpsEOLat java.net.URL.(Unknown Source)EOLat java.net.URL.(Unknown Source)EOLat java

標籤 : powershell windows 10 pid 
詢問時間 : 2022-11-15
瀏覽次數 : 175
內文 : EOL很感謝，[#johncoc]，協助我解決EOL關於Powershell 取得「Win10 應用程序 PID」 (感謝 johncoc 協助解決) 的問題EOL原先我想取得PID的目的是EOL判斷使用者開啟某些網頁，這邊先以Google為例EOL當使用者開啟Google後，開啟指定應用程式，取得PID，然後利用Win10排程，定時去偵測這個PID是否存活。EOL當使用者關閉Google網頁後，判定PID已經消失，則把指定應用程式關閉EOL↑ 這是可行的。EOL但昨天我就在想，想了一天，有沒有什麼方法，可以把監測這功能也寫在PowerShell裡面，就像Ping一樣EOL已經嘗試過的方式有EOLEOL無限迴圈EOLEOL$PIDNu = Get-Process msedge| Where-Object { $_.MainWindowTitle -like '*Google*' } | Select IdEOLEOLFor (;;) {EOL  If ($PIDNu.Id -lt 0) {EOL    Stop-Process -Name chtnode -ForceEOL    BreakEOL  }EOLEOLEOL這沒辦法判斷成功EOLEOL使用 netstat -anoEOLEOLEOL但似乎沒有可以一直監測的效果EOL


標題 : 各位大大好，請教一下用Django & ngrok架站後用liff 網址帶參數卻出現錯誤的問題，謝謝
標籤 : liff.state liff django 
詢問時間 : 2022-11-15
瀏覽次數 : 126
內文 : EOL各位大大好，我用Django設定網頁，用ngrok架站測試帶入參數EOLviews.py檔設定為EOLdef index(request,word1,word2):EOLpost_title = ''EOLreturn render(request,'IT_help/index.html',locals())EOLurls.py檔的urlpatterns 設定為EOLpath('str:word1&str:word2',views.index),EOL呈現網頁的htm

In [15]:
import json #載入json模組
test = open('JSONcrawler.json',encoding="utf8") #開檔
Data = json.load(test) #反序列化(decode)
#把蒐集到的資料印出來
temp = Data.get('Article') #取得 key = "Title" 的 value 
temp #印出 key = "Title" 的 value

[{'標題': '請問java如何獲取字串裡的數字?',
  '標籤': ['android', 'java'],
  '詢問時間': '2022-11-15',
  '瀏覽次數': '26',
  '內文': 'EOL請問java如何獲取字串裡的數字?EOL字串中有英文和數字和＜＞＝符號夾雜，我只要將數字取出放到ｉｎｔ變數裡面請問要如何做？EOL拜託各位了\u3000我上網找只找到一堆不能用的方法\u3000＝\u3000＝\u3000EOL（另外是android studio的ｊａｖａ喔）EOL'},
 {'標題': ' ubuntu的掛載問題',
  '標籤': ['ubuntu 22.04', 'ubuntu', 'linux', 'boot'],
  '詢問時間': '2022-11-15',
  '瀏覽次數': '63',
  '內文': 'EOL根這個狀況蠻類似的EOLhttps://stackoverflow.com/questions/68124511/booting-error-dev-sda2-clean-files-blocksEOL但是我的是 /dev/sdb2EOL雖然這樣開的了機，但是，銀幕亮度卻沒辦法條。EOL請問有什麼辦法可以解決，還有請教一下原因，謝謝。（上面的方法已經試過了）EOL'},
 {'標題': 'Postman Mock Server 可否帶入 Body 參數',
  '標籤': ['#postman', 'body', 'mockserver'],
  '詢問時間': '2022-11-15',
  '瀏覽次數': '69',
  '內文': 'EOL自己實作與查看網路上的文章後，EOL發現 Postman 的 MockServer 好像只能判斷 QueryString 參數，EOL而從 Header、Body 內帶入的參數就完全沒判斷，EOL若有後者的需求，是否就只能自己建立 MockServer 了呢？EOL'},
 {'標題': '試用Apache Jmeter 5.5時，Java不認識https？',
  '標籤': ['apache jmeter'],
  '詢問時間': '2022-11-15',
  '瀏覽次數': '85',
  '內文': 'EOLjava.net.MalformedURLExcep

In [16]:
#印出不同"標籤"出現次數
dic = {} #紀錄下面取得的不同"標籤"出現次數
for i in temp: #從 list 中一個一個列舉出 dict
    #print(li['標籤'])
    for j in i['標籤']: #列舉出每個 key = "標籤" 的 value
        dic[j] = dic.get(j,0)+1 #檢查"標籤"中的每個字串是否出現在 dic 中。如果沒出現，則加入 dic 裡，並使其 value=1 ; 否則已經出現就將其 value+1
dic #利用dic印出不同"標籤"出現次數

{'android': 1,
 'java': 2,
 'ubuntu 22.04': 1,
 'ubuntu': 1,
 'linux': 2,
 'boot': 1,
 '#postman': 1,
 'body': 1,
 'mockserver': 1,
 'apache jmeter': 1,
 '切版': 1,
 'css': 1,
 'dns': 1,
 'bind': 1,
 '@爬蟲': 1,
 '@selenum': 1,
 '#angular': 1,
 '#frontend': 1,
 'robot framework': 1,
 'selenium2library': 1,
 'fortigate': 1,
 '文字編輯器': 1,
 '文字編輯軟體': 1,
 '編輯器': 1,
 '編輯軟體': 1,
 'jquery': 1,
 '轉接': 1,
 'poe': 1,
 '#python求救': 2,
 '#python': 2,
 'telegraf': 1,
 'influxdb': 1,
 'grafana': 1,
 'synology nas': 1,
 '監控': 1,
 'line notify': 1,
 'app': 1,
 'line': 1,
 'excel vba': 1,
 'excel圖表問題': 1,
 '圖表': 1,
 'excel': 1,
 'powershell': 1,
 'windows 10': 1,
 'pid': 1,
 'liff.state': 1,
 'liff': 1,
 'django': 1,
 '資安規範': 1,
 '#anaconda': 1,
 '#jupyter notebook': 1,
 '#新手求指導': 1,
 'react-app-rewired': 1,
 'react': 1,
 'config-overrides.js': 1,
 'bundle.js': 1,
 'nginx': 1,
 'node.js': 1,
 'udp': 1,
 'load balancing': 1,
 'docker': 1,
 '隨身碟': 1,
 'hashtag': 1,
 '部落格': 1,
 'tag': 1,
 'blog': 1,
 '標籤': 1,
